Google of Quotes

First we will import what we will need for this program

In [4]:
import re
from collections import Counter
from pprint import pprint
import math

Our first function is read_quotes().  This will take a text file and read it into a list with the quote and the speaker contained in the same item.  The items will be strings.

In [5]:
def read_quotes(input):

    lines = open(input).read().split('\n')
    quote_list = []
    for i in range(0,len(lines)-1,2):
        quote1 = str(lines[i].replace('"', ""))
        quote1 = str(quote1.replace('.', ""))
        quote1 = str(quote1.replace(',', ""))
        quote1 = str(quote1.replace('?', ""))
        quote1 = str(quote1.replace("(", ""))
        quote1 = str(quote1.replace(")", ""))
        quote1 = str(quote1.replace(":", ""))
        quote1 = str(quote1.replace(";", ""))
        quote2 = quote1 + " - " + str(lines[i+1])
        quote_list.append(quote2)
    return quote_list

The next function is words from quotes.  This function will split a quote into a list of words contained in the quote.  If a word appears in the quote more than once, it will only be in the list once.

In [6]:
def words_from_quotes(input1):

    input1 = input1.lower()
    input1 = str(input1.replace('"', ""))
    input1 = str(input1.replace('.', ""))
    input1 = str(input1.replace(',', ""))
    input1 = str(input1.replace('?', ""))
    input1 = str(input1.replace("(", ""))
    input1 = str(input1.replace(")", ""))
    input1 = str(input1.replace(":", ""))
    input1 = str(input1.replace(";", ""))
    input1 = str(input1.replace("'", "`"))
    word_list = re.split(" ",input1)
    return word_list


creates a dictionary where the keys are full quotes and the values are dictionaries themselves with a key being a word and the value being the number of times the word occurs in the full quote

In [7]:
def create_postings_list_dictionary(input2):

    postings_list = {}
    counter_list = []
    for x in input2:
        counter_dict = {}
        wordlist = words_from_quotes(x)
        for word in wordlist:
            counter = 0
            in_counter_list = False
            for word2 in wordlist:
                if word == word2:
                    counter += 1
            for y in counter_list:
                if word == y:
                    in_counter_list = True
            if in_counter_list == False and word != '-':
                counter_dict[word] = counter
        postings_list[x] = counter_dict
    return postings_list


The function reverse_postings_list creates a list of all words used, then creates a reverse posting list by iterating through all words.

In [8]:
def reverse_postings_list(input3):

    word_list = []
    keys = input3.keys()

# create list of all words used

    for x in keys:
        y = words_from_quotes(x)
        for words in y:
            if words in word_list:
                pass
            elif words != '-':
                word_list.append(words)

#create reverse posting list by iterating through all words
    word_dict = {}

    for x in word_list:
        word_dict[x] = {}
        for y in keys:
            if x in input3[y]:
                counter = input3[y][x]
                word_dict[x][y] = counter

    return word_dict

The function tf takes in a word and a quote and returns the tf*idf value of that word in the given quote

In [9]:
def tf(word, quote):

        max = Counter(quote.split()).most_common()[0][1]
        counter = 0
        word_list = words_from_quotes(quote)
        for x in word_list:
            if x == word:
                counter += 1
        inputnum = counter
        tf = float(inputnum) / float(max)
        quotelist = read_quotes("quotes.txt")
        postlist = create_postings_list_dictionary(quotelist)
        reverse_postlist = reverse_postings_list(postlist)
        idf = math.log(895.0 / float(len(reverse_postlist[word])))
        return idf*tf

The tf_idf function does not answer a particular part of the assignment but is used internally to create a dictionary with keys being the phrases and values being the tf*idf values.  It is passed in a reverse postlist

In [10]:
def tf_idf(input4):

    tf_idf_dict = {}
    for keys in input4.keys():
        max = Counter(keys.split()).most_common()[0][1]
        inputnum = int(input4[keys])
        tf = float(inputnum)/float(max)
        idf = math.log(895.0/float(len(input4)))
        tf_idf_dict[keys] = float(tf*idf)
    return tf_idf_dict


The function one_word_quote_search takes in a single word and outputs a dictionary with keys being the phrases and the values being the tf_idf values

In [11]:
def one_word_quote_search(word):

    quotelist = read_quotes("quotes.txt")
    postlist = create_postings_list_dictionary(quotelist)
    reverse_postlist = reverse_postings_list(postlist)
    tf_dict = tf_idf(reverse_postlist[word])
    return tf_dict

The function mult_words_quote_search takes in a string of undetermined length, cleans it up and splits it into words and returns a dictionary with the sums of the tf-idf for all words in each applicable quote

In [12]:
def mult_words_quote_search(words):

    #takes in a string of undetermined length, splits it into words and returns a dictionary with the sums of the tf-idf
    #for each applicable quote

    quotelist = read_quotes("quotes.txt")
    postlist = create_postings_list_dictionary(quotelist)
    reverse_postlist = reverse_postings_list(postlist)
    words_dict = {}
    word_list = words_from_quotes(words)
    good_word_list = []
    for x in range(len(word_list)):
        if word_list[x] in reverse_postlist.keys():
            words_dict[word_list[x]] = one_word_quote_search(word_list[x])
            good_word_list.append(word_list[x])
    final_dict = {}
    for key in words_dict.keys():
        final_dict.update(words_dict[key])
    for key in final_dict:
        sums = 0
        for y in good_word_list:
            sums += tf(y,key)
        final_dict[key] = sums
    pprint(final_dict)
    return final_dict

Now we just need to run the mult_words_quote_search with user input

In [13]:
mult_words_quote_search(str(raw_input("Input Search: ")))

Input Search: feet
{'A radical is a man with both feet firmly planted in the air - Franklin D. Roosevelt': 4.850913569219542,
 'I cried because I had no shoes `till I met a man who had no feet So I said `You got any shoes you`re not using` - Steven Wright': 1.2127283923048855,
 'If my children wake up on Christmas morning and have someone to thank for putting candy in their stocking have I no one to thank for putting two feet in mine - GK Chesterton': 2.425456784609771,
 'Is there anything more beautiful than a beautiful beautiful flamingo flying across in front of a beautiful sunset And he`s carrying a beautiful rose in his beak and also he`s carrying a very beautiful painting with his feet And also you`re drunk - Jack Handey': 0.8084855948699237,
 'Karate is a form of marital arts in which people who have had years and years of training can using only their hands and feet make some of the worst movies in the history of the world - Dave Barry': 1.2127283923048855,
 'Show me your hands

{'A radical is a man with both feet firmly planted in the air - Franklin D. Roosevelt': 4.850913569219542,
 'I cried because I had no shoes `till I met a man who had no feet So I said `You got any shoes you`re not using` - Steven Wright': 1.2127283923048855,
 'If my children wake up on Christmas morning and have someone to thank for putting candy in their stocking have I no one to thank for putting two feet in mine - GK Chesterton': 2.425456784609771,
 'Is there anything more beautiful than a beautiful beautiful flamingo flying across in front of a beautiful sunset And he`s carrying a beautiful rose in his beak and also he`s carrying a very beautiful painting with his feet And also you`re drunk - Jack Handey': 0.8084855948699237,
 'Karate is a form of marital arts in which people who have had years and years of training can using only their hands and feet make some of the worst movies in the history of the world - Dave Barry': 1.2127283923048855,
 'Show me your hands Do they have scars

All done!